<a href="https://colab.research.google.com/github/DARAKU17/JARVIS/blob/main/JARVIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Path to your local Drive folder where phi-2 is stored
model_path = "/content/drive/MyDrive/phi-2"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load model from local weights
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",       # puts it on GPU if available
    torch_dtype="auto",      # picks correct precision (float16 if GPU supports)
    trust_remote_code=True   # safe for custom architectures
)

# Quick test
inputs = tokenizer("Hello, I'm Phi-2 running from Drive!", return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/drive/MyDrive/phi-2'. Use `repo_type` argument if needed.

In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes -q

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset

model_path = "/content/drive/MyDrive/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype="auto")

# LoRA config
lora_config = LoraConfig(
    r=16,              # rank (higher = more trainable params)
    lora_alpha=16,    # scaling
    target_modules=["q_proj","v_proj"],  # attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# Wrap model
model = get_peft_model(model, lora_config)

In [ ]:
from datasets import Dataset

train_data = {
    "text": [
        "Question: What is 2+2? Answer: 4",
        "Question: What is the capital of Japan? Answer: Tokyo",
        "Question: Who created Roblox? Answer: David Baszucki",
    ]
}

dataset = Dataset.from_dict(train_data)
# Set pad token to eos if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load dataset
dataset = load_dataset("imdb")

# Load tokenizer
model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # important for phi-2

# Tokenize
def tokenize_function(examples):
    tokens = tokenizer(examples["text"], padding="max_length", truncation=True)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Remove raw text column
tokenized_dataset = tokenized_dataset.remove_columns(["text"])

# Split
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]

In [ ]:
from transformers import AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

# Setup LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # works for most transformer LMs
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)
model = get_peft_model(model, lora_config)


# 🔹 Define train/eval datasets
train_dataset = tokenized_dataset["train"]
eval_dataset  = tokenized_dataset["test"]   # IMDB has a test split

# Training args
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./phi2_lora_imdb",
    do_eval=True,               # older flag
    save_strategy="epoch",      # if not supported, remove it
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    fp16=True,
    push_to_hub=False,
)

In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model = AutoModelForCausalLM.from_pretrained(
      model_path,
      torch_dtype="auto",  # or torch.float16 if GPU supports
      device_map=None      # keep fully on GPU (or CPU if no GPU)
      ),
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)
trainer.train()
model.save_pretrained("./phi2_lora_adapter")

In [ ]:
pip install peft